In [1]:
import joblib
import pandas as pd

dataset = "dbpedia"
model_name = f"{dataset}_model"
attack = "paraphrased"
output_file_name = f"artifacts/{dataset}/{model_name}/{dataset}_examples_with_closest_protos_{attack}.csv"
data = joblib.load(
    open(f"artifacts/{dataset}/{model_name}/best_protos_per_testeg.joblib", "rb")
)

FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/dbpedia/dbpedia_model/best_protos_per_testeg.joblib'

In [125]:
original_data_test = pd.read_csv(f"../datasets/{dataset}_dataset/test_{attack}.csv")
original_data_adv = pd.read_csv(f"../datasets/{dataset}_dataset/adv_{attack}.csv")

original_data_adv.head()

,text,label
0,Luis de Quixano y Font served as Mayor of Ponc...,0
1,"In the 1936 college football season, Marquette...",6
2,Victorville's KVTR is a commercial radio stati...,0
3,The Paulinerkirche was a church in Leipzig tha...,2
4,"Moskovia Airlines, located in Zhukovsky, Mosco...",0


In [126]:
test_tb, adv_tb = data[f"test_{attack}"], data[f"adv_{attack}"]
test_protos, test_dist = test_tb[0], test_tb[1]
adv_protos, adv_dist = adv_tb[0], adv_tb[1]

In [127]:
top_test_protos = test_protos[:, 0]
top_test_protos_dist = test_dist[:, 0]

top_adv_protos = adv_protos[:, 0]
top_adv_protos_dist = adv_dist[:, 0]

In [128]:
original_data_test["closest_prototype"] = top_test_protos
original_data_test["closest_prototype_dist"] = top_test_protos_dist
original_data_test["permuted_text"] = original_data_adv["text"]
original_data_test["permuted_closest_prototype"] = top_adv_protos
original_data_test["permuted_closest_prototype_dist"] = top_adv_protos_dist

In [129]:
original_data_test["prototype_changed"] = (
    original_data_test["closest_prototype"]
    != original_data_test["permuted_closest_prototype"]
)
original_data_test.head()

,text,label,closest_prototype,closest_prototype_dist,permuted_text,permuted_closest_prototype,permuted_closest_prototype_dist,prototype_changed
0,"Luis de Quixano y Font was Mayor of Ponce, Pue...",0,8,-3.805425,Luis de Quixano y Font served as Mayor of Ponc...,4,-3.824329,True
1,The 1936 Marquette Golden Avalanche football t...,6,9,-3.659856,"In the 1936 college football season, Marquette...",9,-3.661452,False
2,KVTR is a commercial radio station in Victorvi...,0,5,-3.817369,Victorville's KVTR is a commercial radio stati...,10,-3.345438,True
3,The Paulinerkirche was a church on the Augustu...,2,2,-3.776621,The Paulinerkirche was a church in Leipzig tha...,1,-3.803165,True
4,"Moskovia Airlines (Russian: Авиакомпания \""Мос...",0,8,-3.808982,"Moskovia Airlines, located in Zhukovsky, Mosco...",10,-3.346972,True


In [130]:
original_data_test.to_csv(output_file_name, index=False)